<a href="https://colab.research.google.com/github/Wahesh/threatInspect/blob/main/DownloadDailyMessages_v3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Most of the code for downloading the messages comes from TelegramScrap https://colab.research.google.com/drive/1lzn_XomUI9uCMLkGjQf6-2JnkKazgevh?usp=sharing#scrollTo=NEn8Clkqe0dh

In [ ]:
# @title **1. [ Required ] Set up your credentials once** { display-mode: "form" }

# @markdown Here, you need to input your credentials: `username`, `phone`, `api_id`, and `api_hash`. Your `api_id` and `api_hash` can only be generated from [Telegram's app creation page](https://my.telegram.org/apps). If you want to get the English translations, you also need the Gemini API key. Once your credentials are set up, you won’t need to update them again. Just click “Run” to proceed.

# Install the Telethon library for Telegram API interactions
!pip install -q telethon

# Initial imports
from datetime import datetime, timezone
import pandas as pd
import time
import json
import re

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
# Telegram imports
from telethon.sync import TelegramClient

# Google Colab imports
from google.colab import files

#Gemini AI imports
from google.api_core import retry
from google.generativeai.types import RequestOptions

import google.generativeai as genai

#imports for formatting Excel
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
from openpyxl.styles import  Border, Side, PatternFill, Alignment

# Setup / change only the first time you use it
# @markdown **1.1.** Your Telegram account username (just 'abc123', not '@'):
username = 'none' # @param {type:"string"}
# @markdown **1.2.** Your Telegram account phone number (ex: '+9011999999999'):
phone = '+90111111111' # @param {type:"string"}
# @markdown **1.3.** Telegram API ID, it can be only generated from https://my.telegram.org/apps:
api_id = '11111' # @param {type:"string"}
# @markdown **1.4.** Telegram API hash, also from https://my.telegram.org/apps:
api_hash = '1a1a1a1a1aa1a1a1a1a1a1a1a1a1aa1a1a1a1a' # @param {type:"string"}
# @markdown **1.5** Gemini API key, from Google AI studio https://ai.google.dev/gemini-api/docs/api-key. If you do not want to translate the messages automatically, you can leave this field with the default value. **In such case remember to select "no" in point 2.4 below!**:
gemini_key = '1a1a1a1a1aa1a1a1a1a1a1a1a1a1aa1a1a1a1a' # @param {type:"string"}

In [ ]:
# @title **2. [ Required ] Adjust every time you want to use it** { display-mode: "form" }

# @markdown In this section, you will define the parameters for scraping data from Telegram channels or groups. Specify the channels you want to scrape using the format `https://t.me/ChannelName`. Do not use URLs starting with `https://web.telegram.org/`. Set the date range by defining the start and end day, month, and year. Choose an output file name for the scraped data. Decide if you want to also get English translations using Gemini AI (the program will run longer in this case).

# Setup / change every time to define scraping parameters

# @markdown **2.1.** Here you put the name of the channel or group that you want to scrape, e.g.: 'https://t.me/zoda_gov_ua'. **Just write the `channel names` always separated by commas (,) without spaces:**
channels = 'https://t.me/zoda_gov_ua,https://t.me/mykolaivskaODA,https://t.me/Sumy_news_ODA,https://t.me/mykola_lukashuk,https://t.me/synegubov,https://t.me/khmelnytskaODA,https://t.me/odeskaODA,https://t.me/kozytskyy_maksym_official,https://t.me/VadymFilashkin,https://t.me/KyivCityOfficial,https://t.me/VA_Kyiv,https://t.me/poltavskaOVA,https://t.me/kirovogradskaODA,https://t.me/zhytomyrskaODA,https://t.me/mod_russia,https://t.me/kpszsu,https://t.me/Ukrenergo' # @param {type:"string"}
channels = [channel.strip() for channel in channels.split(",")]

# @markdown **2.2.** Here you can select the `time window` you would like to extract data from the listed channels:
date_min = '2024-12-23' # @param {type:"date"}
date_max = '2024-12-31' # @param {type:"date"}

date_min = datetime.fromisoformat(date_min).replace(tzinfo=timezone.utc)
date_max = datetime.fromisoformat(date_max).replace(tzinfo=timezone.utc)

# @markdown **2.3.** Choose a `name` for the final file you want to download as output. If you want to keep the file in the default format, please leave "default" in the field below:
file_name = 'default' # @param {type:"string"}
if file_name=="default":
  file_name= 'daily_messages'
# @markdown **2.4.** Would you like to translate the messages to English? (the Gemini APi_key in point 1.5 must be filled out correctly in this case)
translate_english = "yes"  # @param ["yes", "no"]

# @markdown **2.6.** Would you like to automatically add information where possible? (if you select no, the spreadsheed will contain only downloaded messages in Ukraininan and English, remaining values will be empty)
automatic_info = "yes"  # @param ["yes", "no"]

# @markdown **2.7.** Choose the format of the final file you want to download.
File = 'excel' # @param ["excel", "csv"]

max_t_index = 1000000
time_limit = 21600
key_search = ''
if translate_english=="yes":
  genai.configure(api_key=gemini_key)

In [ ]:
 #@title **3. [ Required ] Start Telegram scraping** { display-mode: "form" }

# @markdown **Attention:** During this step, Telegram may request a verification code. Please monitor your Telegram app and input the required information promptly. Rest assured, all data entered remains secure.
# @markdown The program will let you know when it finishes each step: scraping data, translating, and filling out automatic fields (if you have selected such options).
# @markdown The files will then download automatically.

data = []  # List to store scraped data
t_index = 0  # Tracker for the number of messages processed
start_time = time.time()  # Record the start time for the scraping session

# Function to remove invalid XML characters from text
def remove_unsupported_characters(text):
    valid_xml_chars = (
        "[^\u0009\u000A\u000D\u0020-\uD7FF\uE000-\uFFFD"
        "\U00010000-\U0010FFFF]"
    )
    cleaned_text = re.sub(valid_xml_chars, '', text)
    return cleaned_text

# Function to format time in days, hours, minutes, and seconds
def format_time(seconds):
    days = seconds // 86400
    hours = (seconds % 86400) // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f'{int(days):02}:{int(hours):02}:{int(minutes):02}:{int(seconds):02}'

# Function to print progress of the scraping process
def print_progress(t_index, message_id, start_time, max_t_index):
    elapsed_time = time.time() - start_time
    current_progress = t_index / (t_index + message_id) if (t_index + message_id) <= max_t_index else t_index / max_t_index
    percentage = current_progress * 100
    estimated_total_time = elapsed_time / current_progress
    remaining_time = estimated_total_time - elapsed_time

    elapsed_time_str = format_time(elapsed_time)
    remaining_time_str = format_time(remaining_time)

    print(f'Progress: {percentage:.2f}% | Elapsed Time: {elapsed_time_str} | Remaining Time: {remaining_time_str}')

#Function to check if the channel is one of the regional reports or other
def is_report(channel_adress):
  regional_reports = [
    'https://t.me/zoda_gov_ua',
    'https://t.me/olexandrprokudin',
    'https://t.me/mykolaivskaODA',
    'https://t.me/Sumy_news_ODA',
    'https://t.me/mykola_lukashuk',
    'https://t.me/synegubov',
    'https://t.me/khmelnytskaODA',
    'https://t.me/odeskaODA',
    'https://t.me/kozytskyy_maksym_official',
    'https://t.me/VadymFilashkin',
    'https://t.me/KyivCityOfficial',
    'https://t.me/VA_Kyiv',
    'https://t.me/poltavskaOVA',
    'https://t.me/kirovogradskaODA',
    'https://t.me/zhytomyrskaODA']
  if channel_adress in regional_reports:
    return "yes"
  else:
    return "no"

#Function to get the name of channel (have to be pre-set)
def channel2name(channel_adress):
  name_dict = {
    'https://t.me/zoda_gov_ua':'Zaporizhzhia',
    'https://t.me/olexandrprokudin':'Kherson',
    'https://t.me/mykolaivskaODA':'Mykolaiv',
    'https://t.me/Sumy_news_ODA':'Sumy',
    'https://t.me/mykola_lukashuk':'Dnipropetrovsk',
    'https://t.me/synegubov':'Kharkiv',
    'https://t.me/khmelnytskaODA':'Khmelnytskyi',
    'https://t.me/odeskaODA':'Odesa',
    'https://t.me/kozytskyy_maksym_official':'Lviv',
    'https://t.me/VadymFilashkin':'Donetsk',
    'https://t.me/KyivCityOfficial':'Kyiv',
    'https://t.me/VA_Kyiv':'Kyiv 2',
    'https://t.me/poltavskaOVA':'Poltava',
    'https://t.me/kirovogradskaODA':'Kirovohrad',
    'https://t.me/zhytomyrskaODA':'Zhytomyr',
    'https://t.me/mod_russia':'Russia Ministry of Defence',
    'https://t.me/kpszsu':'Ukraine Air Defence',
    'https://t.me/Ukrenergo':'Ukraine Energy Company'
  }
  if channel_adress in name_dict:
    return name_dict[channel_adress]
  else:
    return "none"

def check_area_info(source_channels):
  #define dictionaries for regions
  #security area dictionary
  srm_dict = {"unknown":"unknown","volyn":"western","lviv":"western","zakarpattia":"western","ivano-frankivsk":"western","ternopil":"western","rivne":"western",
               "chernivtsy":"western","khmelnytsky":"western","zhytomyr":"western","vinnytsia":"western","kyiv":"western","cherkasy":"western",
              "kropyvnytskyi":"western","poltava":"western","west chernihiv":"western","odesa":"southern","mykolaiv":"southern","kherson":"southern",
              "sumy":"eastern","kharkiv":"eastern","dnipro":"eastern","zaporizhzhia":"eastern","donetsk":"eastern","north chernihiv":"eastern"}
  #channel to region dictionary
  region_dict = {
    'https://t.me/zoda_gov_ua':'Zaporizhzhia',
    'https://t.me/olexandrprokudin':'Kherson',
    'https://t.me/mykolaivskaODA':'Mykolaiv',
    'https://t.me/Sumy_news_ODA':'Sumy',
    'https://t.me/mykola_lukashuk':'Unknown',
    'https://t.me/synegubov':'Kharkiv',
    'https://t.me/khmelnytskaODA':'Khmelnytsky',
    'https://t.me/odeskaODA':'Odesa',
    'https://t.me/kozytskyy_maksym_official':'Lviv',
    'https://t.me/VadymFilashkin':'Donetsk',
    'https://t.me/KyivCityOfficial':'Kyiv',
    'https://t.me/VA_Kyiv':'Kyiv',
    'https://t.me/poltavskaOVA':'Poltava',
    'https://t.me/kirovogradskaODA':'Kropyvnytskyi',
    'https://t.me/zhytomyrskaODA':'Zhytomyr',
    'https://t.me/mod_russia':'Unknown',
    'https://t.me/kpszsu':'Unknown',
    'https://t.me/Ukrenergo':'Unknown'
  }
  #define columns for security area and region

  region = [region_dict[ch] if ch in region_dict else "unknown" for ch in source_channels]
  assert len(region)==len(source_channels)
  security_area = [srm_dict[reg.lower()] if reg.lower() in srm_dict else "unknown" for reg in region]
  assert len(security_area)==len(source_channels)
  return security_area, region

#function to split datetime into two df columns
def convert_datetime(datetime_list):
  date_list = []
  time_list = []
  for dt in datetime_list:
    date, time = dt.split(" ")
    date_list.append(date)
    time_list.append(time)
  return date_list, time_list

#funtion to assign target and perpetrator based on the channel name
def assign_target_perpetrator(channel_source):
  target_dict = {
    'https://t.me/zoda_gov_ua':'Ukraine',
    'https://t.me/olexandrprokudin':'Ukraine',
    'https://t.me/mykolaivskaODA':'Ukraine',
    'https://t.me/Sumy_news_ODA':'Ukraine',
    'https://t.me/mykola_lukashuk':'Ukraine',
    'https://t.me/synegubov':'Ukraine',
    'https://t.me/khmelnytskaODA':'Ukraine',
    'https://t.me/odeskaODA':'Ukraine',
    'https://t.me/kozytskyy_maksym_official':'Ukraine',
    'https://t.me/VadymFilashkin':'Ukraine',
    'https://t.me/KyivCityOfficial':'Ukraine',
    'https://t.me/VA_Kyiv':'Ukraine',
    'https://t.me/poltavskaOVA':'Ukraine',
    'https://t.me/kirovogradskaODA':'Ukraine',
    'https://t.me/zhytomyrskaODA':'Ukraine',
    'https://t.me/mod_russia':'Unknown',
    'https://t.me/kpszsu':'Unknown',
    'https://t.me/Ukrenergo':'Unknown'
  }

  perp_dict = {
    'https://t.me/zoda_gov_ua':'Russia',
    'https://t.me/olexandrprokudin':'Russia',
    'https://t.me/mykolaivskaODA':'Russia',
    'https://t.me/Sumy_news_ODA':'Russia',
    'https://t.me/mykola_lukashuk':'Russia',
    'https://t.me/synegubov':'Russia',
    'https://t.me/khmelnytskaODA':'Russia',
    'https://t.me/odeskaODA':'Russia',
    'https://t.me/kozytskyy_maksym_official':'Russia',
    'https://t.me/VadymFilashkin':'Russia',
    'https://t.me/KyivCityOfficial':'Russia',
    'https://t.me/VA_Kyiv':'Russia',
    'https://t.me/poltavskaOVA':'Russia',
    'https://t.me/kirovogradskaODA':'Russia',
    'https://t.me/zhytomyrskaODA':'Russia',
    'https://t.me/mod_russia':'Unknown',
    'https://t.me/kpszsu':'Unknown',
    'https://t.me/Ukrenergo':'Unknown'
  }

  target = [target_dict[ch] if ch in target_dict else "Unknown" for ch in channel_source]
  perpetrator = [perp_dict[ch] if ch in target_dict else "Unknown" for ch in channel_source]
  return target, perpetrator

#Function to translate Ukrainian messages to English

def get_gemini_translation(df_all,input_column):
  model = genai.GenerativeModel("gemini-1.5-flash")
  translations = []
  prompt = 'Translate the text to English: '
  untranslated_list = df[input_column].tolist()
  for i,ukr_text in enumerate(untranslated_list):
    if i%25==0:
      print(f'Finished translating {i} out of {len(untranslated_list)} messages')
    completed_prompt = prompt+ukr_text
    model_answer = model.generate_content(completed_prompt, request_options=RequestOptions(
                                        retry=retry.Retry(
                                            initial=10,
                                            multiplier=2,
                                            maximum=60,
                                            timeout=300)))
    translations.append(model_answer.text)
  assert len(untranslated_list)==len(translations)
  df["Text_english"] = translations
  return df


# Normalize File variable to avoid issues
File = re.sub(r'[^a-z]', '', File.lower())  # Converts to lowercase and removes non-alphabetic characters

# Scraping process
for channel in channels:
    if t_index >= max_t_index:
        break

    if time.time() - start_time > time_limit:
        break

    loop_start_time = time.time()

    try:
        c_index = 0
        async with TelegramClient(username, api_id, api_hash) as client:
            async for message in client.iter_messages(channel, search=key_search):
                try:
                    if date_min <= message.date <= date_max:

                        # Process comments of the message
                        comments_list = []
                        try:
                            async for comment_message in client.iter_messages(channel, reply_to=message.id):
                                comment_text = comment_message.text.replace("'", '"')

                                comment_media = 'True' if comment_message.media else 'False'

                                comment_emoji_string = ''
                                if comment_message.reactions:
                                    for reaction_count in comment_message.reactions.results:
                                        emoji = reaction_count.reaction.emoticon
                                        count = str(reaction_count.count)
                                        comment_emoji_string += emoji + " " + count + " "

                                comment_date_time = comment_message.date.strftime('%Y-%m-%d %H:%M:%S')

                                comments_list.append({
                                    'Type': 'comment',
                                    'Comment Group': channel,
                                    'Comment Author ID': comment_message.sender_id,
                                    'Comment Content': comment_text,
                                    'Comment Date': comment_date_time,
                                    'Comment Message ID': comment_message.id,
                                    'Comment Author': comment_message.post_author,
                                    'Comment Views': comment_message.views,
                                    'Comment Reactions': comment_emoji_string,
                                    'Comment Shares': comment_message.forwards,
                                    'Comment Media': comment_media,
                                    'Comment Url': f'https://t.me/{channel}/{message.id}?comment={comment_message.id}'.replace('@', ''),
                                })
                        except Exception as e:
                            comments_list = []
                            print(f'Error processing comments: {e}')

                        # Process the main message
                        media = 'True' if message.media else 'False'

                        emoji_string = ''
                        if message.reactions:
                            for reaction_count in message.reactions.results:
                                emoji = reaction_count.reaction.emoticon
                                count = str(reaction_count.count)
                                emoji_string += emoji + " " + count + " "

                        date_time = message.date.strftime('%Y-%m-%d %H:%M:%S')
                        cleaned_content = remove_unsupported_characters(message.text)
                        cleaned_comments_list = remove_unsupported_characters(json.dumps(comments_list))

                        data.append({
                            'Group': channel,
                            'Group_name': channel2name(channel),
                            'Date': date_time,
                            'Text_ukrainian': cleaned_content
                        })

                        c_index += 1
                        t_index += 1

                        # Print progress
                        print(f'{"-" * 80}')
                        print_progress(t_index, message.id, start_time, max_t_index)
                        current_max_id = min(c_index + message.id, max_t_index)
                        print(f'From {channel}: {c_index:05} contents of {current_max_id:05}')
                        print(f'Id: {message.id:05} / Date: {date_time}')
                        print(f'Total: {t_index:05} contents until now')
                        print(f'{"-" * 80}\n\n')

                        if t_index % 1000 == 0:
                            if File == 'parquet':
                                backup_filename = f'backup_{file_name}_until_{t_index:05}_{channel}_ID{message.id:07}.parquet'
                                pd.DataFrame(data).to_parquet(backup_filename, index=False)
                            elif File == 'excel':
                                backup_filename = f'backup_{file_name}_until_{t_index:05}_{channel}_ID{message.id:07}.xlsx'
                                pd.DataFrame(data).to_excel(backup_filename, index=False, engine='openpyxl')

                        if t_index >= max_t_index:
                            break

                        if time.time() - start_time > time_limit:
                            break

                    elif message.date < date_min:
                        break

                except Exception as e:
                    print(f'Error processing message: {e}')

        print(f'\n\n##### {channel} was ok with {c_index:05} posts #####\n\n')

        df = pd.DataFrame(data)
        if File == 'parquet':
            partial_filename = f'complete_{channel}_in_{file_name}_until_{t_index:05}.parquet'
            df.to_parquet(partial_filename, index=False)
        elif File == 'excel':
            partial_filename = f'complete_{channel}_in_{file_name}_until_{t_index:05}.xlsx'
            df.to_excel(partial_filename, index=False, engine='openpyxl')
        # files.download(partial_filename)

    except Exception as e:
        print(f'{channel} error: {e}')

    loop_end_time = time.time()
    loop_duration = loop_end_time - loop_start_time

    if loop_duration < 60:
        time.sleep(60 - loop_duration)
df = pd.DataFrame(data)
df = df[df["Text_ukrainian"]!='']
print(f'\n{"-" * 50}\n#Finished scraping! #{t_index:05} posts were scraped!\n{"-" * 50}\n\n\n\n')

#add translations to the dataframe
if translate_english=="yes":
  df = get_gemini_translation(df,"Text_ukrainian")
  print(f'\n{"-" * 50}\n#Finished translating!\n\n\n\n')

#add the columns to DataFrame to make it into template

if automatic_info=="no":
  df["security_area"] = "none"
  df["region"] = "none"
  df["city/town/area"] = "none"
  df["date of event"], df["time of event"] = convert_datetime(df["Date"].tolist())
  df["source message (original)"] = df["Text_ukrainian"]
  df["source message (translated)"] = df["Text_english"]
  df["target group"] = "none"
  df["perpetrator group"] = "none"
  df["threat_type"] = "none"
  df["incident type"] = "none"
  df["no of IEDs, mortars, etc."] = "none"
  df["analysis/comments"] = "none"
  df["total casualities"] = 0
  df["deaths"] = 0
  df["injuries"] = 0
  df["source channel"] = df["Group"]

else:
  df["security_area"],df["region"] = check_area_info(df["Group"].tolist())
  df["city/town/area"] = "none"
  df["date of event"], df["time of event"] = convert_datetime(df["Date"].tolist())
  df["source message (original)"] = df["Text_ukrainian"]
  df["source message (translated)"] = df["Text_english"]
  df["target group"], df["perpetrator group"] = assign_target_perpetrator(df["Group"].tolist())
  df["threat_type"] = "none"
  df["incident type"] = "none"
  df["no of IEDs, mortars, etc."] = "none"
  df["analysis/comments"] = "none"
  df["total casualities"] = 0
  df["deaths"] = 0
  df["injuries"] = 0
  df["source channel"] = df["Group"]

#sort by telegram channel and the date (oldest messages on the top)
df = df.sort_values(['source channel','date of event','time of event'])

#drop unnecessary columns
df = df.drop('Group', axis=1)
df = df.drop('Group_name', axis=1)
df = df.drop('Date', axis=1)
df = df.drop('Text_ukrainian', axis=1)
df = df.drop('Text_english', axis=1)

#format Excel file

#function for border settings
def set_border(ws, cell_range):
    thin = Side(border_style="thin", color="000000")
    for row in ws[cell_range]:
        for cell in row:
            cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

#copy DataFrame data to an Excel spreadsheet
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(df, index=False, header=True):
    ws.append(r)

#format first row as column titles
ws.print_title_rows='1:1'
ws.freeze_panes = "A2"

fill1 = PatternFill(start_color="00FFFFCC",end_color="00FFFFCC",fill_type='solid')
fill2 = PatternFill(start_color="00CCFFFF",end_color="00CCFFFF",fill_type='solid')
fill3 = PatternFill(start_color="00CCFFCC",end_color="00CCFFCC",fill_type='solid')
fill4 = PatternFill(start_color="00FFCC99",end_color="00FFCC99",fill_type='solid')
fill5 = PatternFill(start_color="00FFFF99",end_color="00FFFF99",fill_type='solid')
fill6 = PatternFill(start_color="009999FF",end_color="009999FF",fill_type='solid')


ws["A1"].fill = fill1
ws["B1"].fill = fill1
ws["C1"].fill = fill1
ws["D1"].fill = fill2
ws["E1"].fill = fill2
ws["F1"].fill = fill3
ws["G1"].fill = fill3
ws["H1"].fill = fill4
ws["I1"].fill = fill4
ws["J1"].fill = fill5
ws["K1"].fill = fill5
ws["L1"].fill = fill5
ws["M1"].fill = fill6
ws["N1"].fill = fill1
ws["O1"].fill = fill1
ws["P1"].fill = fill1
ws["Q1"].fill = fill2



#validate security area
sc_list = sorted(["unknown","western","southern","eastern"])
dv_sc = DataValidation(type="list", formula1=f'"{",".join(sc_list)}"', showDropDown=False, allow_blank=True)
ws.add_data_validation(dv_sc)
dv_sc.add('A1:A1048576')

#validate region
re_list = sorted(["Unknown","Volyn","Lviv","Zakarpattia","Ivano-frankivsk","Ternopil","Rivne","Chernivtsy","Khmelnytsky","Zhytomyr","Vinnytsia","Kyiv","Cherkasy","Kropyvnytskyi",
           "Poltava","West Chernihiv","Odesa","Mykolaiv","Kherson","Sumy","Kharkiv","Dnipro","Zaporizhzhia","Donetsk","North Chernihiv"])
dv_re = DataValidation(type="list", formula1=f'"{",".join(re_list)}"', showDropDown=False, allow_blank=True)
ws.add_data_validation(dv_re)
dv_re.add('B1:B1048576')


#validate taget and perpetrator
tp_list = ["Ukraine","Russia"]
dv_tp = DataValidation(type="list", formula1=f'"{",".join(tp_list)}"', showDropDown=False, allow_blank=True)
ws.add_data_validation(dv_tp)
dv_tp.add('H1:H1048576')


p_list = ["Ukraine","Russia"]
dv_p = DataValidation(type="list", formula1=f'"{",".join(p_list)}"', showDropDown=False, allow_blank=True)
ws.add_data_validation(dv_p)
dv_p.add('I1:I1048576')

#validate the threat type
th_list = sorted(["Armed Conflict", "Terrorism", "Crime", "Civil Unrest", "Hazard", "Information"])
dv_th = DataValidation(type="list", formula1=f'"{",".join(th_list)}"', showDropDown=False, allow_blank=True)
ws.add_data_validation(dv_th)
dv_th.add('J1:J1048576')


#wrap text in cells
for row in ws.iter_rows():
    for cell in row:
        cell.alignment = Alignment(horizontal='center', vertical='center',wrapText=True)

#add border to header
set_border(ws, 'A1:Q1')


#save the Excel file
final_filename = file_name+".xlsx"
wb.save(final_filename)

